<a href="https://www.kaggle.com/code/alistnel/pytorch-neural-crf?scriptVersionId=240347497" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# This is NER Training Notes

We can use LSTM wia: https://github.com/aliustunelin/pytorch_neural_crf

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/turkish-ner-dataset/trainer.py
/kaggle/input/turkish-ner-dataset/ner_predictor.py
/kaggle/input/turkish-ner-dataset/transformers_predictor.py
/kaggle/input/turkish-ner-dataset/README.md
/kaggle/input/turkish-ner-dataset/transformers_trainer_ddp.py
/kaggle/input/turkish-ner-dataset/transformers_trainer.py
/kaggle/input/turkish-ner-dataset/preprocess/get_elmo_vec.py
/kaggle/input/turkish-ner-dataset/preprocess/get_bert_vec.py
/kaggle/input/turkish-ner-dataset/docs/benchmark.md
/kaggle/input/turkish-ner-dataset/docs/fast_crf.md
/kaggle/input/turkish-ner-dataset/docs/other_usage.md
/kaggle/input/turkish-ner-dataset/docs/transformers_benchmark.md
/kaggle/input/turkish-ner-dataset/src/__init__.py
/kaggle/input/turkish-ner-dataset/src/model/neuralcrf.py
/kaggle/input/turkish-ner-dataset/src/model/transformers_neuralcrf.py
/kaggle/input/turkish-ner-dataset/src/model/__init__.py
/kaggle/input/turkish-ner-dataset/src/model/embedder/word_embedder.py
/kaggle/input/turkish-ner-dataset

In [2]:
!git clone https://github.com/aliustunelin/pytorch_neural_crf.git
import os
os.chdir('pytorch_neural_crf') 
!ls

Cloning into 'pytorch_neural_crf'...
remote: Enumerating objects: 959, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 959 (delta 263), reused 289 (delta 210), pack-reused 591 (from 1)
Receiving objects: 100% (959/959), 4.02 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (576/576), done.
data		  README.md		     transformers_trainer_ddp.py
docs		  src			     transformers_trainer.py
ner_predictor.py  trainer.py
preprocess	  transformers_predictor.py


In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # Kaggle GPU'ları ile uyumlu PyTorch versiyonu
!pip install transformers
!pip install datasets
!pip install accelerate # If necessary (for distributed training)
!pip install seqeval

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 1

In [4]:
!ls -all
!cd data/Turkish-Wiki-NER-Dataset && ls -all

total 100
drwxr-xr-x 7 root root  4096 May 18 03:28 .
drwxr-xr-x 3 root root  4096 May 18 03:28 ..
drwxr-xr-x 5 root root  4096 May 18 03:28 data
drwxr-xr-x 2 root root  4096 May 18 03:28 docs
drwxr-xr-x 8 root root  4096 May 18 03:28 .git
-rw-r--r-- 1 root root  8098 May 18 03:28 ner_predictor.py
drwxr-xr-x 2 root root  4096 May 18 03:28 preprocess
-rw-r--r-- 1 root root  5624 May 18 03:28 README.md
drwxr-xr-x 5 root root  4096 May 18 03:28 src
-rw-r--r-- 1 root root 13337 May 18 03:28 trainer.py
-rw-r--r-- 1 root root  4038 May 18 03:28 transformers_predictor.py
-rw-r--r-- 1 root root 17320 May 18 03:28 transformers_trainer_ddp.py
-rw-r--r-- 1 root root 16378 May 18 03:28 transformers_trainer.py
total 3688
drwxr-xr-x 2 root root    4096 May 18 03:28 .
drwxr-xr-x 5 root root    4096 May 18 03:28 ..
-rw-r--r-- 1 root root  185617 May 18 03:28 dev.txt
-rw-r--r-- 1 root root     883 May 18 03:28 README.md
-rw-r--r-- 1 root root  194107 May 18 03:28 test.txt
-rw-r--r-- 1 root root 3376628

In [5]:
!mkdir /kaggle/input/alistnel-turkish-ner/data
!cp -r /kaggle/input/alistnel-turkish-ner/data ./data/Turkish-Wiki-NER-Dataset

mkdir: cannot create directory ‘/kaggle/input/alistnel-turkish-ner/data’: No such file or directory
cp: cannot stat '/kaggle/input/alistnel-turkish-ner/data': No such file or directory


In [6]:
!git pull
!git checkout alistnel/add-ruff-formatter
!git branch

Already up to date.
Branch 'alistnel/add-ruff-formatter' set up to track remote branch 'alistnel/add-ruff-formatter' from 'origin'.
Switched to a new branch 'alistnel/add-ruff-formatter'
* alistnel/add-ruff-formatter
  master


In [7]:
        #Update transformers_util to fix adamw transform method error
        with open("/kaggle/working/pytorch_neural_crf/src/config/transformers_util.py", "r") as f:
            lines = f.readlines()

        # refactor adamW to transform utils
        new_lines = []
        replaced = False
        for line in lines:
            if "from transformers import AdamW, get_linear_schedule_with_warmup" in line and not replaced:
                new_lines.append("from torch.optim import AdamW\n")
                new_lines.append("from transformers import get_linear_schedule_with_warmup\n")
                replaced = True
            else:
                new_lines.append(line)

        with open("/kaggle/working/pytorch_neural_crf/src/config/transformers_util.py", "w") as f:
            f.writelines(new_lines)

        print("updated transformers_util.py")

updated transformers_util.py


In [8]:
        #Update config>utils to fix adamw transform method error
        file_path = "/kaggle/working/pytorch_neural_crf/src/config/utils.py"
        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        replaced = False
        for line in lines:
            if "from transformers import AdamW" in line and not replaced: 
                new_lines.append("from torch.optim import AdamW\n")
                replaced = True
                print(f"Line replaced in {file_path}")
            else:
                new_lines.append(line)
        
        if replaced:
            with open(file_path, "w") as f:
                f.writelines(new_lines)
            print(f"{file_path} has been updated.")
        else:
            print(f"Target import line not found in {file_path}. Please check the file content.")

Line replaced in /kaggle/working/pytorch_neural_crf/src/config/utils.py
/kaggle/working/pytorch_neural_crf/src/config/utils.py has been updated.


In [9]:
!python transformers_trainer.py \
    --device=cuda:0 \
    --dataset=/data/data/Turkish-Wiki-NER-Dataset \
    --model_folder=/kaggle/working/saved_modells \
    --embedder_type=bert-base-multilingual-cased \
    --num_epochs=3 \
    --batch_size=16 # TODO: this is test batch size

2025-05-18 03:31:09.251736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747539069.443705     114 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747539069.496780     114 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
tokenizer_config.json: 100%|██████████████████| 49.0/49.0 [00:00<00:00, 334kB/s]
config.json: 100%|█████████████████████████████| 625/625 [00:00<00:00, 5.44MB/s]
vocab.txt: 100%|█████████████████████████████| 996k/996k [00:00<00:00, 7.80MB/s]
tokenizer.json: 100%|██████████████████████| 1.96M/1.96M [00:00<00:00, 26.0MB/s]
Traceback (most recent call last):
  File "/kaggle/working/pytorch_neural_crf/transformers_trainer.py", line 477,

In [10]:
!ls -al

total 100
drwxr-xr-x 7 root root  4096 May 18 03:28 .
drwxr-xr-x 3 root root  4096 May 18 03:28 ..
drwxr-xr-x 5 root root  4096 May 18 03:28 data
drwxr-xr-x 2 root root  4096 May 18 03:28 docs
drwxr-xr-x 8 root root  4096 May 18 03:30 .git
-rw-r--r-- 1 root root  8098 May 18 03:28 ner_predictor.py
drwxr-xr-x 2 root root  4096 May 18 03:28 preprocess
-rw-r--r-- 1 root root  5624 May 18 03:28 README.md
drwxr-xr-x 6 root root  4096 May 18 03:30 src
-rw-r--r-- 1 root root 13337 May 18 03:28 trainer.py
-rw-r--r-- 1 root root  4038 May 18 03:28 transformers_predictor.py
-rw-r--r-- 1 root root 17320 May 18 03:28 transformers_trainer_ddp.py
-rw-r--r-- 1 root root 16378 May 18 03:28 transformers_trainer.py
